##Run All
Enter necessary information in Startup [OPENAI_API_KEY, OPENAI_ORGANIZATION_ID] then press Ctrl+F9 (Windows) or Toolbar > Runtime > Run All

In [ ]:
#@title Startup
import os
%pip install openai flask flask_socketio
from verifier.verify import verify_xdl
with open("XDL_description.txt", "r") as f:
  XDL_description = f.read()
os.environ["OPENAI_API_KEY"] = "Enter OPENAI API KEY"
os.environ["OPENAI_ORGANIZATION_ID"] = "Enter OPENAI ORG ID"

: 

In [ ]:
#@title XDL Generation
import os
import openai

def prompt(instructions, description, max_tokens, model="text-davinci-003"):
    if "Enter" in os.environ["OPENAI_API_KEY"]:
      raise Exception("Missing API_Key or ORG_ID")

    """Function that calls the OpenAI API"""
    if model == "text-davinci-003":
        response = openai.Completion.create(
            model="text-davinci-003",
            prompt=description + "\nConvert to XDL:\n" + instructions,
            temperature=0,
            max_tokens=max_tokens,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
        )
        return response["choices"][0]["text"]

    elif model == "gpt-3.5-turbo" or model == "gpt-4":
        if "Enter" in os.environ["OPENAI_ORGANIZATION_ID"]:
          raise Exception("Missing API_Key or ORG_ID")
        response = openai.ChatCompletion.create(
            model=model,
            messages = [
                {"role":"system", "content":"You are a natural language to XDL translator, you must also do your best to correct any incorrect XDL, only use items contained in the description, here is a description of XDL:\n"+description },
                {"role":"user", "content": f"\nConvert/Correct the following to proper XDL:\n{instructions}"}
            ],
            temperature=0,
            max_tokens=max_tokens,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
        )
        return response["choices"][0]["message"]["content"]


def translate(input_xdl, model):
    """Function that translates the input XDL"""

    openai.api_key = os.environ["OPENAI_API_KEY"]
    openai.organization = os.environ["OPENAI_ORGANIZATION_ID"]

    correct_syntax = False
    errors = {}
    prev_input_xdl = input_xdl

    # Start 10 iteration for loop to limit token usage
    for step in range(10):
        try:
            gpt3_output = prompt(input_xdl, XDL_description, 1000, model)
        except Exception as e:
            print(f"\u001b[31m{e}.\033[0m Too many tokens required or invalid API key.")
            break
        compile_correct = verify_xdl(gpt3_output)
        errors[step] = {
            "errors": compile_correct,
            "input_xdl": input_xdl,
            "gpt3_output": gpt3_output,
        }
        if not compile_correct:
            correct_syntax = True
            break
        else:
            error_list = set()
            for item in compile_correct:
                for error in item["errors"]:
                    error_list.add(error)
            error_message = f"\n{gpt3_output}\nThis XDL was not correct. These were the errors\n{os.linesep.join(list(error_list))}\nPlease fix the errors."
            input_xdl = f"{prev_input_xdl} {error_message}"

        print("gpt3_output:::")
        print(error_message, "\n")
    try:
        if correct_syntax:
            xdl = gpt3_output
        else:
            xdl = "The correct XDL could not be generated."

    except Exception as e:
        print(f"Error: {e}")

    print(f"XDL: {xdl}")
    print(f"Final syntax valid: {correct_syntax}")
    return xdl

def main():
    print("Welcome to the XDL Translator CLI!")

    input_xdl = input("Enter the text to be converted to XDL: ")
    model = input("Choose the GPT model version (text-davinci-003, gpt-3.5-turbo, or gpt-4): ")

    translated_xdl = translate(input_xdl, model)

    print(translated_xdl)


if __name__ == "__main__":
    main()
